In [3]:
import numpy as np
import librosa
import extract_audiofeatures as ea

In [4]:
wav_in = '../sourcedata/data/HCP_7T_movie_FIX/stimuli/7T_MOVIE1_CC1_v2.wav'

In [5]:
y,sr,dur_10hz = ea.load_wav(wav_in)

In [6]:
dur_10hz

9210

In [9]:
chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=1600)

In [10]:
chroma.shape

(12, 9211)